# Import libraries


In [4]:
import os
import sys
import subprocess
import tqdm
from typing import List, Tuple

# Verify The Total TXL Can Parse
function named `parse_unparse_contracts` is used to parse and unparse contracts using TXL.
The function takes two arguments:
- `txl`: the path to the TXL file
- `paths`: a list of paths to the contracts to parse
The function returns two lists:
- `outputs`: a list of paths to the contracts that were parsed
- `errors`: a list of paths to the contracts that were not parsed

for errors is a list of paths to the contracts that were not parsed.
for outputs is a list of paths to the contracts that were parsed.
$$TXL_{errors} = |errors|$$
$$TXL_{outputs} = |outputs|$$
$$TXL_{total} = TXL_{errors} + TXL_{outputs}$$
$$TXL_{ratio} = \frac{TXL_{outputs}}{TXL_{total}}$$
$$TXL_{\bar{ratio}} = \frac{TXL_{errors}}{TXL_{total}}$$

In [5]:
def parse_unparse_contracts(txl: str, paths: list) -> Tuple[List[str], List[str]]:
    """
    Parse and unparse contracts.
    using TXL
    :return:
    """
    errors = []
    outputs = []
    # for each file in the contracts folder
    # do the command 'txl ?.sol ?.txl'
    for path in paths:
        # get the files in the contracts folder
        for p in os.listdir(path):
            if p.endswith('.sol'):
                ps = os.path.abspath(os.path.join(path, p))
                # get errors and output from txl
                out, err = (subprocess.Popen(['txl', txl, ps], stdout=subprocess.PIPE, stderr=subprocess.PIPE)
                            .communicate())
                tqdm.tqdm.write('Parsing {}'.format(ps))
                # if there are errors
                if err:
                    tqdm.tqdm.write('Errors:')
                    tqdm.tqdm.write(err.decode('utf-8'))
                    errors.append(ps)
                # if there is output
                if out:
                    tqdm.tqdm.write('Output:')
                    tqdm.tqdm.write(out.decode('utf-8'))
                    outputs.append(ps)

    # show progress bar showing the number of files parsed and did not parse
    tqdm.tqdm.write('Parsed: {}'.format(len(outputs)))
    tqdm.tqdm.write('Errors: {}'.format(len(errors)))
    tqdm.tqdm.write('Total: {}'.format(len(outputs) + len(errors)))
    # ratio of files parsed to files not parsed
    tqdm.tqdm.write('Ratio Parsed: {}'.format(len(outputs) / (len(outputs) + len(errors))))
    tqdm.tqdm.write('Ratio ~Parsed: {}'.format(len(errors) / (len(outputs) + len(errors))))
    return outputs, errors

# Parse and Unparse Contracts (get all the contracts)

In [6]:
# get the path to the contracts
contracts_path = [os.path.abspath(os.path.join(os.getcwd(), '..', 'scripts', 'custom_contracts')),
                    os.path.abspath(os.path.join(os.getcwd(), '..', 'scripts', 'gpt_contracts')),
                    os.path.abspath(os.path.join(os.getcwd(), '..', 'scripts', 'github_contracts', 'error_contracts')),
                    os.path.abspath(os.path.join(os.getcwd(), '..', 'scripts', 'github_contracts', 'verified_contracts')),
                  ]

contracts_path

['/Users/sourenakhanzadeh/programming/txl/SolOSphere/scripts/custom_contracts',
 '/Users/sourenakhanzadeh/programming/txl/SolOSphere/scripts/gpt_contracts',
 '/Users/sourenakhanzadeh/programming/txl/SolOSphere/scripts/github_contracts/error_contracts',
 '/Users/sourenakhanzadeh/programming/txl/SolOSphere/scripts/github_contracts/verified_contracts']

In [8]:
# get the path to the txl file
txl_path = os.path.abspath(os.path.join(os.getcwd(), '..', 'SolO', 'txl', 'new_version', 'optimize.txl'))

txl_path

'/Users/sourenakhanzadeh/programming/txl/SolOSphere/SolO/txl/new_version/optimize.txl'

In [9]:
# parse and unparse the contracts
outputs, errors = parse_unparse_contracts(txl_path, contracts_path)

Parsing /Users/sourenakhanzadeh/programming/txl/SolOSphere/scripts/custom_contracts/mutate41.sol
Errors:
TXL v10.8b (13.7.22) (c) 1988-2022 Queen's University at Kingston
Compiling /Users/sourenakhanzadeh/programming/txl/SolOSphere/SolO/txl/new_version/optimize.txl ... 
Parsing /Users/sourenakhanzadeh/programming/txl/SolOSphere/scripts/custom_contracts/mutate41.sol ...
Transforming ...

Output:
pragma solidity ^ 0.8 .0; contract mutated41 {
    int public counter;
    int public x = 10;
    uint [] public array;
    constructor () {
        counter = 10 * 20 * 30 * 56 * 56;
    }
    function loop24 () public {
        for (uint256 i = 0; i < 24; i + +) {
            counter + = 9;
        }
    }
    function loop25 (int [] memory arr) public {
        for (uint256 i = 0; i < arr.length; i + +) {
            x + = arr [i] * 2;
        }
    }
}

Parsing /Users/sourenakhanzadeh/programming/txl/SolOSphere/scripts/custom_contracts/mutate9.sol
Errors:
TXL v10.8b (13.7.22) (c) 1988-2022 Qu